# Segmenting and Clustering Neighborhoods in Toronto

### Installing dependencies

In [8]:
import sys
!conda install --yes --prefix {sys.prefix} numpy
!conda install --yes --prefix {sys.prefix} pandas
!conda install --yes --prefix {sys.prefix} lxml

print('Dependencies installed')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Dependencies installed


### Importing libraries

In [19]:
import numpy as np
import pandas as pd
print('Libraries imported')

Libraries imported


### Reading HTML with pandas

In [20]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url)[0]
print(df.shape)
df.head()

(180, 3)


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Only process the cells that have an assigned borough.
Ignore cells with a borough that is Not assigned.

In [21]:
df_cleared = df[df['Borough'] != 'Not assigned']
df_cleared = df_cleared.reset_index()
df_cleared = df_cleared.drop('index', axis=1)
print(df_cleared.shape)
df_cleared.head()

(103, 3)


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### If a cell has a borough but a Not assigned neighborhood.
Then the neighborhood will be the same as the borough.   
Let's check

In [22]:
df_cleared[df_cleared['Neighborhood' ] == 'Not assigned']

,Postal code,Borough,Neighborhood


### More than one neighborhood can exist in one postal code area.
Rows will be combined into one row with the neighborhoods separated with a comma.   

Let's check if one or more postal code have more than one neighborhood

In [28]:
print(len(df_cleared['Postal code']) == len(np.unique(df_cleared['Postal code'])))

True


In [27]:
df_cleared['Postal code'].value_counts()

M4W    1
M4H    1
M3K    1
M5P    1
M3N    1
      ..
M5K    1
M1G    1
M3H    1
M4M    1
M8X    1
Name: Postal code, Length: 103, dtype: int64

In [31]:
df_cleared.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [30]:
df_cleared.shape


(103, 3)